In [1]:
from utils import convert2coco
from utils import dataset_split
from utils import dataset_analysis

In [2]:
base = './data/ori'
annotations = convert2coco(base)

In [3]:
split_dictionary = {
    'train': 0.60,
    'val': 0.20,
    'test': 0.20
}

ann_split = dataset_split(annotations, split_dictionary, 10000)

The best error: 0.0015675939908796877
The best error: 0.0013362167678603618
The best error: 0.0011080581421764363
The best error: 0.0006965233343231244
The best error: 0.0003417283004523244
The best error: 0.00019286006529224358
The best error: 0.0001758275875362365
The best error: 0.00016364124241476308
The best error: 3.216095039177539e-05
The best error: 1.5650759700978796e-05


In [4]:
results = dataset_analysis(ann_split)

-----------------------------------
num_images         180
num_objects      23139
-----------------------------------
num_images on each set

train         108    0.600
val            36    0.200
test           36    0.200
-----------------------------------
num_objects on each set

train       13908    0.601
val          4650    0.201
test         4581    0.198
-----------------------------------
Category: 1

train        6806    0.600
val          2275    0.201
test         2257    0.199
-----------------------------------
Category: 2

train        7102    0.602
val          2375    0.201
test         2324    0.197
-----------------------------------
